sets up gdrive


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')#, force_remount=True)
root_dir = "/content/gdrive/My Drive/"

the following three lines ensure that any edits to libraries you make are reloaded here automatically, and also that any charts or images displayed are shown in this notebook.

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

 install secure connection library

In [ ]:
!pip install 'pyOpenSSL>=0.11'

     |████████████████████████████████| 61kB 2.4MB/s 
     |████████████████████████████████| 3.2MB 4.0MB/s 


In [ ]:
!pip install earthengine-api 

authenticate earthengine to work with your google account

In [ ]:
!earthengine authenticate

Instructions for updating:
non-resource variables are not supported in the long term
To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=4MNpH73Fjmj1KmEhTSJ7u3KX9v5_8_MadQlYiTk7_o8&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AY0e-g4wJMHLKS5RHBDGxk6oa0V_TXP86jtsgxSFF2FQUbvAiqIWtpsuAN4

Successfully saved authorization token.


In [ ]:
import ee; ee.Initialize();

test correct earthengine setup for notebook, if no error then ok :)

In [ ]:
!pip install -U -q PyDrive

sets up gdrive within earthengine for input and output

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

import csv from gdrive

In [ ]:
#paste into link the link of csv file on your gdrive
import pandas as pd
# Cambiar el id, "Obtener link"
link = 'https://drive.google.com/open?id=1ZCw1sStEBJIVwaDmh8iCQm0pM-Xo-Fnv'
fluff, id = link.split('=')
print (id) # Verify that you have everything after '='
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('CNN_CENI_COL_79979_ALL.csv')  
df3 = pd.read_csv('CNN_CENI_COL_79979_ALL.csv')

#cnn_col_2018_annual_month_median_n

1ZCw1sStEBJIVwaDmh8iCQm0pM-Xo-Fnv


check sample of csv

In [ ]:
df3.shape

(77979, 3)

In [ ]:
# Set the id = rownumber as index of the DataFrame
df3 = df3.set_index('id')
df3 = df3.sort_values(by=['id'])
df3.head()

,lat,long
id,,
1,-4.191180,-69.973730
2,-4.191180,-69.939235
3,-4.156684,-69.973730
4,-4.156684,-69.939235
5,-4.122189,-70.008225


In [ ]:
#SENTINEL export

import ee

ee.Initialize()

##################################
#manually created bounding polygon for Colombia

#COLOMBIA
tha_shp = ee.Geometry.Polygon(
        [[[-78.9909352282, 12.4373031682],
          [-78.9909352282, -4.29818694419],
          [-66.8763258531, -4.29818694419],
          [-66.8763258531, 12.4373031682]]]);

#SENTINEL START
#/**
# * Function to mask clouds using the Sentinel-2 QA band
# * @param {ee.Image} image Sentinel-2 image
# * @return {ee.Image} cloud masked Sentinel-2 image
# */
def maskS2clouds(image):
    qa = image.select('QA60')
#  // Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
#  // Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))
    return image.updateMask(mask).divide(10000)


#// Map the function over one year of data and take the median.
#// Load Sentinel-2 TOA reflectance data.

rgbVis = {'min': 0.0,'max': 0.3,'bands': ['B4', 'B3', 'B2'],}

dataset = ee.ImageCollection('COPERNICUS/S2').filterDate('2020-01-01', '2020-12-31').filterBounds(tha_shp).filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 60)).map(maskS2clouds)

upres = dataset.median().visualize(**rgbVis)

#################################

#downloaded image is black -> switch coordinates :)

#all bands must have the same data type when exporting, you should download only the bands you need and cast them to a type that suit all bands.
##################################

# gives a tuple of column name and series 
# for each column in the dataframe 
vector_faltante = [170,2910,8000,22729,22730,22731,22732,22733,22734,22735,22736,22737,22738,22739,22740,22741,22742,22743,22744,22745,22746,22747,22748,22749,22750,22751,22752,22753,22754,22755,22756,22757,22758,22759,22760,22761,22762,22763,22764,22765,22766,22767,22768,29140,29258,29269,29302,29314,29321,29329,29362,29366,29370,29377,29382,29383,29398,29401,29407,29416,29425,29432,29436,29438,29444,29445,29447,29449,29450,29453,29456,29461,29465,29471,29474,29476,29477,29478,29479,29483,29485,29489,29499,29505,29510,29517,29519,29520,29526,29534,29535,29536,29545,29548,29549,29553,29562,29564,29565,29567,29569,29572,29583,29593,29595,29605,29624,29628,29629,29637,29639,29645,29646,29649,29652,29657,29663,29664,29665,29673,29675,29677,29680,29692,29696,29705,29708,29711,29713,29715,29733,29735,29739,29748,29767,29768,29776,29787,29794,29796,29804,29807,29808,29809,29811,29819,29823,29839,29843,29844,29845,29846,29847,29860,29863,29867,29872,29875,29877,29880,29886,29895,29897,29906,29907,29918,29919,29922,29933,29935,29939,29943,29947,29948,29950,29955,29964,29993,29997,30005,30015,30022,30024,30033,30036,30051,30056,30059,30063,30067,30070,30072,30074,30075,30093,30097,30098,30101,30102,30106,30115,30119,30129,30131,30134,30138,30141,30150,30151,30168,30177,30179,30181,30189,30201,30206,30217,30218,30225,30236,30237,30252,30262,30263,30264,30265,30268,30269,30272,30274,30276,30277,30286,30287,30289,30290,30291,30294,30300,30306,30308,30319,30321,30325,30326,30332,30341,30345,30349,30353,30354,30357,30365,30366,30369,30372,30374,30375,30376,30378,30381,30389,30398,30410,30419,30422,30438,30448,30453,30455,30456,30460,30463,30465,30466,30471,30476,30477,30478,30480,30493,30503,30504,30514,30517,30520,30522,30529,30530,30532,30542,30543,30553,30564,30579,30583,30584,30590,30592,30595,30608,30617,30623,30625,30629,30632,30634,30636,30640,30641,30643,30645,30648,30650,30658,30660,30661,30662,30678,30680,30684,30688,30696,30701,30706,30712,30714,30716,30718,30730,30738,30739,30745,30766,30767,30775,30790,30793,30810,30825,30845,30847,30852,30853,30854,30856,30860,30874,30879,30882,30888,30890,30891,30894,30903,30908,30912,30922,30925,30929,30933,30937,30939,30940,30942,30945,30947,30948,30952,30955,30961,30966,30977,30988,30992,30997,31013,31016,31019,31021,31022,31023,31024,31031,31033,31034,31037,31039,31053,31060,31064,31072,31079,31081,31083,31085,31087,31090,31093,31108,31113,31117,31118,31120,31126,31127,31134,31136,31137,31143,31153,31156,31157,31159,31162,31166,31174,31176,31179,31191,31193,31196,31203,31206,31207,31211,31212,31215,31217,31219,31220,31221,31227,31229,31232,31233,31234,31238,31241,31245,31246,31254,31266,31279,31280,31284,31286,31293,31294,31301,31306,31309,31312,31314,31318,31322,31325,31337,31342,31343,31347,31348,31357,31363,31364,31365,31369,31371,31372,31373,31393,31396,31397,31399,31405,31407,31412,31414,31433,31434,31439,31448,31449,31451,31455,31465,31466,31469,31471,31472,31473,31484,31487,31493,31497,31501,31514,31523,31531,31547,31550,31557,31562,31565,31566,31570,31574,31575,31581,31584,31589,31592,31599,31614,31615,31616,31622,31623,31625,31637,31640,31643,31645,31646,31648,31650,31654,31659,31666,31668,31670,31674,31679,31680,31689,31692,31699,31700,31703,31709,31710,31711,31714,31715,31720,31727,31732,31734,31737,31740,31749,31755,31756,31759,31764,31778,31783,31785,31787,31790,31802,31806,31808,31809,31824,31836,31849,31850,31855,31858,31863,31864,31870,31874,31878,31879,31884,31886,31887,31908,31914,31915,31933,31934,31936,31937,31939,31942,31944,31952,31953,31955,31957,31959,31962,31963,31970,31973,31977,31981,31986,31987,31992,31998,31999]

for i in vector_faltante:
  #if i > 70193 and i < 71029 :  
    c_lon = df3['long'][i]
    c_lat = df3['lat'][i]
    geometry = ee.Geometry.Point([c_lon, c_lat]).buffer(1920)#1920metres because we want resolution of image 384x384 pixels -> 10x384/2 = 1920
    geometry = geometry.getInfo()['coordinates'][0]
    task_config = {
        'scale': 10,
        'region': geometry,
        'driveFolder': 'CNN_IMG_COL_2020_3840_TIF_79979_ALL_FALTANTE',
    }

    task = ee.batch.Export.image(upres, 'CNN_DIMG_COL_2020_ST_384_3840_13_' + '{:06d}'.format(i), task_config)
  
        #print('Export Image '+ str(i)+ ' was submitted, please wait ...');

    task.start()